# 02 – Model Development: Deepfake Detection

Goal: extract image frames from videos, build a train/val split, and prepare data
loaders for a CNN based deepfake classifier.

Imports and setup

In [1]:
from pathlib import Path
import json

import pandas as pd
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch version: 2.9.1+cpu
CUDA available: False


Load metadata again

In [3]:
from pathlib import Path
import json
import pandas as pd

# project root is one level up from notebooks/
project_root = Path("..").resolve()

# data/raw under project root
data_root = project_root / "data" / "raw"

meta_path = data_root / "train_sample_videos" / "metadata.json"
print(meta_path, meta_path.exists())   # should print ...metadata.json True

with open(meta_path, "r") as f:
    meta = json.load(f)

rows = []
for fname, info in meta.items():
    rows.append(
        {
            "filename": fname,
            "label": info["label"],          # "FAKE" or "REAL"
            "original": info.get("original") # original real video, if available
        }
    )

df = pd.DataFrame(rows)
df.head(), df["label"].value_counts()

C:\Users\adamc\Documents\Fall 25\Machine and Deep Learning\CSC422_DeepfakeDetection_Final_Aguilar_Adam\data\raw\train_sample_videos\metadata.json True


(         filename label        original
 0  aagfhgtpmv.mp4  FAKE  vudstovrck.mp4
 1  aapnvogymq.mp4  FAKE  jdubbvfswz.mp4
 2  abarnvbtwb.mp4  REAL            None
 3  abofeumbvv.mp4  FAKE  atvmxvwyns.mp4
 4  abqwwspghj.mp4  FAKE  qzimuostzz.mp4,
 label
 FAKE    323
 REAL     77
 Name: count, dtype: int64)

Create processed frame folders

In [4]:
frames_root = Path("data/processed/frames")
train_frames_root = frames_root / "train"

for label in ["REAL", "FAKE"]:
    (train_frames_root / label).mkdir(parents=True, exist_ok=True)

train_frames_root

WindowsPath('data/processed/frames/train')

Helper to extract a center frame from each video

In [5]:
def extract_center_frame(video_path: Path, out_path: Path) -> bool:
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print("Could not open", video_path)
        return False

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count == 0:
        cap.release()
        print("No frames in", video_path)
        return False

    center_idx = frame_count // 2
    cap.set(cv2.CAP_PROP_POS_FRAMES, center_idx)
    ok, frame = cap.read()
    cap.release()

    if not ok:
        print("Could not read frame from", video_path)
        return False

    # OpenCV is BGR, but for saving jpg it is fine to keep BGR
    out_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(out_path), frame)
    return True

Run frame extraction (one per video)

In [6]:
from tqdm.auto import tqdm

success = 0
fail = 0

for row in tqdm(df.itertuples(), total=len(df)):
    fname = row.filename
    label = row.label.upper()          # "REAL" or "FAKE"

    video_path = data_root / "train_sample_videos" / fname
    out_name = fname.replace(".mp4", ".jpg")
    out_path = train_frames_root / label / out_name

    # Skip if we already extracted this frame
    if out_path.exists():
        continue

    if extract_center_frame(video_path, out_path):
        success += 1
    else:
        fail += 1

success, fail

c:\Users\adamc\Documents\Fall 25\Machine and Deep Learning\CSC422_DeepfakeDetection_Final_Aguilar_Adam\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 400/400 [01:49<00:00,  3.67it/s]


(400, 0)